In [1]:
import torch
import torch.nn as nn
import math
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

## RoPEの仕組みをつくる
* `inv_freq`：周波数の逆数。次元ごとに異なる周期を持つ
  $$
  \text{inv_freq}_i = \frac{1}{10000^{2i / d}}
  $$
* `positions`：トークンの位置（0, 1, 2, ...）
* `angles = positions × inv_freq`
  → 各位置・各次元ごとに回転角（ラジアン値）を作る
* `cos(angles)` と `sin(angles)` を事前にキャッシュする


## 処理の意味
* `inv_freq`：周波数の逆数。次元ごとに異なる周期を持つ
  $$
  \text{inv_freq}_i = \frac{1}{10000^{2i / d}}
  $$
* `positions`：トークンの位置（0, 1, 2, ...）
* `angles = positions × inv_freq`
  → 各位置・各次元ごとに回転角（ラジアン値）を作る
* `cos(angles)` と `sin(angles)` を事前にキャッシュする

つまり、**各位置に対応する回転行列（cos/sin）を用意**しているのです。

In [ ]:
# ==========================================
# 🔹 Rotary Position Embedding (RoPE)
# ==========================================

# 位置に応じた回転角を作る
def build_rope_cache(seq_len, dim):
  half_dim = dim // 2
  freq_seq = torch.arange(half_dim, dtype=torch.float32)
  inv_freq = 1.0 / (10000 ** (freq_seq / half_dim))
  positions = torch.arange(seq_len, dtype=torch.float32).unsqueeze(1)
  angles = positions * inv_freq.unsqueeze(0)
  return torch.cos(angles), torch.sin(angles)


In [3]:
dim = 256
seq_len = 64

half_dim = dim // 2
freq_seq = torch.arange(half_dim, dtype=torch.float32)
inv_freq = 1.0 / (10000 ** (freq_seq / half_dim))
positions = torch.arange(seq_len, dtype=torch.float32).unsqueeze(1)
angles = positions * inv_freq.unsqueeze(0)

inv_freq


tensor([1.0000e+00, 9.3057e-01, 8.6596e-01, 8.0584e-01, 7.4989e-01, 6.9783e-01,
        6.4938e-01, 6.0430e-01, 5.6234e-01, 5.2330e-01, 4.8697e-01, 4.5316e-01,
        4.2170e-01, 3.9242e-01, 3.6517e-01, 3.3982e-01, 3.1623e-01, 2.9427e-01,
        2.7384e-01, 2.5483e-01, 2.3714e-01, 2.2067e-01, 2.0535e-01, 1.9110e-01,
        1.7783e-01, 1.6548e-01, 1.5399e-01, 1.4330e-01, 1.3335e-01, 1.2409e-01,
        1.1548e-01, 1.0746e-01, 1.0000e-01, 9.3057e-02, 8.6596e-02, 8.0584e-02,
        7.4989e-02, 6.9783e-02, 6.4938e-02, 6.0430e-02, 5.6234e-02, 5.2330e-02,
        4.8697e-02, 4.5316e-02, 4.2170e-02, 3.9242e-02, 3.6517e-02, 3.3982e-02,
        3.1623e-02, 2.9427e-02, 2.7384e-02, 2.5483e-02, 2.3714e-02, 2.2067e-02,
        2.0535e-02, 1.9110e-02, 1.7783e-02, 1.6548e-02, 1.5399e-02, 1.4330e-02,
        1.3335e-02, 1.2409e-02, 1.1548e-02, 1.0746e-02, 1.0000e-02, 9.3057e-03,
        8.6596e-03, 8.0584e-03, 7.4989e-03, 6.9783e-03, 6.4938e-03, 6.0430e-03,
        5.6234e-03, 5.2330e-03, 4.8697e-

従来の「位置埋め込み（positional encoding）」では、
単語ベクトルに直接 sin/cos 値を **足し合わせて** いました。

一方、RoPE は **回転操作として組み込む** ことで：

* 相対位置の差（Δpos）をベクトルの**回転角の差**として表現できる
* Attention計算中でも位置差が自然に保存される

つまり、**位置の「差分」がAttentionの内積に直接反映**されるのです。